In [1]:
from treys import Evaluator
from treys import Card
from treys import Deck
from utils import lget, setify
import itertools

In [2]:
class Poker:
    def __init__(self, strategy=None, desiredCards=None):
        self.strategy = strategy
        self.desiredCards = (
            [
                Card.print_pretty_card(Card.new(card)) if len(card) > 1 else card
                for card in desiredCards
            ]
            if desiredCards
            else None
        )
        self.POKER_HAND_EVALUATOR = Evaluator()

    def isTypeOfCard(self, card):
        for desiredCard in self.desiredCards:
            if desiredCard in card:
                self.desiredCards.remove(desiredCard)
                return True
        return False

    def simulatePokerStrategy(self):
        playerCards, communityCards, deck = set(), set(), Deck()
        while (desiredCombo := self.isDesiredHand(communityCards)) == None:
            card = deck.draw(1)
            communityCards.add(card)

        communityCards -= setify(desiredCombo)
        communityCards |= setify(deck.draw(5 - len(communityCards)))
        playerCards = desiredCombo

        return list(playerCards), list(communityCards)

    def simulatePokerGeneral(self):
        playerCards, communityCards, deck = [], [], Deck()
        while len(playerCards) != 5 or len(communityCards) < 5:
            card = deck.draw(1)
            if len(playerCards) != 5:
                stringCardDrawn = Card.print_pretty_card(card)
                if self.isTypeOfCard(stringCardDrawn):
                    playerCards.append(card)
                else:
                    communityCards.append(card)

        return playerCards, communityCards

    # tech debt, should be done with polymorphism instead
    def isDesiredHand(self, playerCards):
        allSatisfiedHands, strongestComboIndex = [], 0
        handRankingsUpperBoundsForStrats = {
            "straight": (1609, 1600),
            "flush": (1599, 323),
            "pair": (6185, 3326),
            "two-pair": (3325, 2468),
            "trips": (2467, 1610),
            "boat": (322, 167),
            "quads": (166, 11),
            "royal": (1, 1),
            "straight-flush": (10, 1),
        }

        lowerBound, upperBound = handRankingsUpperBoundsForStrats[self.strategy]

        totalCombinations = list(itertools.combinations(playerCards, 5))
        for combination in totalCombinations:
            handStrength = self.POKER_HAND_EVALUATOR.evaluate(list(combination), [])
            if handStrength >= upperBound and handStrength <= lowerBound:
                allSatisfiedHands.append(combination)

        if allSatisfiedHands:
            (_, strongestComboIndex) = max(
                (self.POKER_HAND_EVALUATOR.evaluate(list(satisfiedHand), []), i)
                for i, satisfiedHand in enumerate(allSatisfiedHands)
            )

        return lget(allSatisfiedHands, strongestComboIndex)

    def isPlayerHandBetter(self, playerCards, communityCards):
        if not playerCards or not communityCards:
            print("ERROR: NO PLAYER CARDS OR NO COMMUNITY CARDS")

        playerHandStrength = self.POKER_HAND_EVALUATOR.evaluate(playerCards, [])
        totalCombinations = list(itertools.combinations(communityCards, 5))
        (communityHandStrength, communityHandComboIndex) = min(
            (self.POKER_HAND_EVALUATOR.evaluate(list(combination), []), index)
            for index, combination in enumerate(totalCombinations)
        )

        return (
            playerHandStrength <= communityHandStrength,
            totalCombinations[communityHandComboIndex],
        )

    def playPoker(self):
        if self.desiredCards and self.strategy:
            print("ERROR: WE CANNOT HAVE BOTH A STRATEGY AND DESIRED CARDS")
        playerCards, communityCards = (
            self.simulatePokerGeneral()
            if self.desiredCards
            else self.simulatePokerStrategy()
        )
        playerWin, bestCommunityCombination = self.isPlayerHandBetter(
            playerCards, communityCards
        )
        return (
            playerWin,
            Card.print_pretty_cards(playerCards),
            Card.print_pretty_cards(communityCards),
            Card.print_pretty_cards(bestCommunityCombination),
        )

In [3]:
poker = Poker(strategy= 'two-pair')

In [4]:
poker.desiredCards

In [5]:
poker.strategy

'two-pair'

In [6]:
poker.playPoker()

(True,
 ' [2♥],[3♣],[3♠],[4♠],[4♦] ',
 ' [K♦],[7♠],[8♠],[9♥],[J♠] ',
 ' [K♦],[7♠],[8♠],[9♥],[J♠] ')

In [7]:
poker = Poker(desiredCards=['Th', 'A', 'K', 'Q', 'J'])


In [8]:
poker.desiredCards

['[T♥]', 'A', 'K', 'Q', 'J']

In [9]:
poker.strategy

In [10]:
poker.playPoker()

(False,
 ' [J♥],[A♣],[K♥],[Q♣],[T♥] ',
 ' [6♥],[6♠],[9♥],[4♦],[4♥],[A♦],[7♥],[K♦],[K♣],[5♣],[J♠],[8♥],[2♣],[7♣],[8♠],[T♦],[9♦],[7♦] ',
 ' [7♥],[K♦],[K♣],[7♣],[7♦] ')